# Anomaly Detection with `h2o`

In [1]:
import h2o
h2o.init()

FILE = './anomaly-5k.csv'

df = h2o.import_file(FILE)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2018-10-16; OpenJDK Runtime Environment (build 11.0.2+7); OpenJDK 64-Bit Server VM (build 11.0.2+7, mixed mode)
  Starting server from /usr/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpb7uqsydo
  JVM stdout: /tmp/tmpb7uqsydo/h2o_kedwin_started_from_python.out
  JVM stderr: /tmp/tmpb7uqsydo/h2o_kedwin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,"7 days, 18 hours and 1 minute"
H2O cluster name:,H2O_from_python_kedwin_8cg9ld
H2O cluster total nodes:,1
H2O cluster free memory:,7.805 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [2]:
seed = 12345
ntrees = 100
isoforest = h2o.estimators.H2OIsolationForestEstimator(
    ntrees=ntrees, seed=seed)
isoforest.train(x=df.col_names[0:31], training_frame=df)
predictions = isoforest.predict(df)

predictions

isolationforest Model Build progress: |███████████████████████████████████| 100%
isolationforest prediction progress: |████████████████████████████████████| 100%


predict,mean_length
0.373134,5.93
0.347015,6
0.25,6.26
0.339552,6.02
0.339552,6.02
0.291045,6.15
0.272388,6.2
0.347015,6
0.395522,5.87
0.391791,5.88


In [3]:
predictions.cor()

predict,mean_length
1,-1
-1,1


In [4]:
quantile = 0.95
quantile_frame = predictions.quantile([quantile])
quantile_frame

Probs,predictQuantiles,mean_lengthQuantiles
0.95,0.350746,6.97


In [5]:
threshold = quantile_frame[0, "predictQuantiles"]
predictions["predicted_class"] = predictions["predict"] > threshold
predictions["class"] = df["Class"]
predictions

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column Class not found
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_4_sid_8cb5 (append predictions_a8f7_IsolationForest_model_python_1549144235511_1_on_anomaly_5k.hex (> (cols_py predictions_a8f7_IsolationForest_model_python_1549144235511_1_on_anomaly_5k.hex 'predict') 0.35074626865671643) 'predicted_class' (cols_py anomaly_5k.hex 'Class') 'class'))", 'session_id': '_sid_8cb5'}


In [9]:
%matplotlib notebook
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
import numpy as np


def get_auc(labels, scores):
    fpr, tpr, thresholds = roc_curve(labels, scores)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score


def get_aucpr(labels, scores):
    precision, recall, th = precision_recall_curve(labels, scores)
    aucpr_score = np.trapz(recall, precision)
    return precision, recall, aucpr_score


def plot_metric(ax, x, y, x_label, y_label, plot_label, style="-"):
    ax.plot(x, y, style, label=plot_label)
    ax.legend()
    
    ax.set_ylabel(x_label)
    ax.set_xlabel(y_label)


def prediction_summary(labels, predicted_score, predicted_class, info, plot_baseline=True, axes=None):
    if axes is None:
        axes = [plt.subplot(1, 2, 1), plt.subplot(1, 2, 2)]

    fpr, tpr, auc_score = get_auc(labels, predicted_score)
    plot_metric(axes[0], fpr, tpr, "False positive rate",
                "True positive rate", "{} AUC = {:.4f}".format(info, auc_score))
    if plot_baseline:
        plot_metric(axes[0], [0, 1], [0, 1], "False positive rate",
                "True positive rate", "baseline AUC = 0.5", "r--")

    precision, recall, aucpr_score = get_aucpr(labels, predicted_score)
    plot_metric(axes[1], recall, precision, "Recall",
                "Precision", "{} AUCPR = {:.4f}".format(info, aucpr_score))
    if plot_baseline:
        thr = sum(labels)/len(labels)
        plot_metric(axes[1], [0, 1], [thr, thr], "Recall",
                "Precision", "baseline AUCPR = {:.4f}".format(thr), "r--")

    plt.show()
    return axes


def figure():
    fig_size = 4.5
    f = plt.figure()
    f.set_figheight(fig_size)
    f.set_figwidth(fig_size*2)


h2o_predictions = predictions.as_data_frame()

figure()
axes = prediction_summary(
    h2o_predictions["class"], h2o_predictions["predict"], h2o_predictions["predicted_class"], "h2o")

H2OServerError: HTTP 500 Server Error:
'{"__meta":{"schema_version":3,"schema_name":"H2OErrorV3","schema_type":"H2OError"},"timestamp":1549142372535,"error_url":"/3/DownloadDataset","msg":"\\n\\nERROR MESSAGE:\\n\\nCaught exception: java.lang.NullPointerException\\n\\n","dev_msg":"\\n\\nERROR MESSAGE:\\n\\nCaught exception: java.lang.NullPointerException from: water.api.DatasetServlet.doGet(DatasetServlet.java:35)\\n\\n","http_status":500,"values":{},"exception_type":"java.lang.NullPointerException","exception_msg":"\\n\\nERROR MESSAGE:\\n\\nnull\\n\\n","stacktrace":["java.lang.NullPointerException","    water.api.DatasetServlet.doGet(DatasetServlet.java:35)","    javax.servlet.http.HttpServlet.service(HttpServlet.java:735)","    javax.servlet.http.HttpServlet.service(HttpServlet.java:848)","    org.eclipse.jetty.servlet.ServletHolder.handle(ServletHolder.java:684)","    org.eclipse.jetty.servlet.ServletHandler.doHandle(ServletHandler.java:501)","    org.eclipse.jetty.server.handler.Co

In [13]:
from sklearn.ensemble import IsolationForest
import pandas as pd

df_pandas = df.as_data_frame()
df_train_pandas = df_pandas.iloc[:, :30]

x = IsolationForest(random_state=seed, contamination=(1-quantile),
                    n_estimators=ntrees, behaviour="new").fit(df_train_pandas)

iso_predictions = x.predict(df_train_pandas)
iso_score = x.score_samples(df_train_pandas)

sk_predictions = pd.DataFrame({
    "predicted_class": list(map(lambda x: 1*(x == -1), iso_predictions)),
    "class": h2o_predictions["class"],
    "predict": -iso_score
})

sk_predictions.head()

ValueError: could not convert string to float: 'FRESHPOINT_158'